In [ ]:
!pip install --upgrade pip

%pip install -U transformers datasets accelerate peft trl bitsandbytes
%pip install google-api-python-client==1.7.2 google-auth==2.14.1 google-auth-httplib2==0.0.3 google-auth-oauthlib==0.4.1
%pip install langgraph==0.0.37
%pip install typing_extensions

!pip install sentencepiece==0.1.99

!pip install html2text
%pip install "huggingface_hub>=0.34.0,<1.0"

RESTART SESSION


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datetime import datetime

In [ ]:
from email.mime.text import MIMEText

In [ ]:
from datetime import datetime

In [ ]:
import os
import json
import base64
import re
import html2text
import time
from datetime import datetime, timedelta

# Google API & Auth
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# LangGraph for the agent structure
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Set, Dict

# Transformers, TRL & PEFT for the AI model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import Dataset, load_dataset




**The Agent State**


In [ ]:
class AgentState(TypedDict, total=False):
    """Defines the memory or state of our agent's workflow."""
    unread_emails: List[Dict]
    current_email_id: str
    current_email_content: str
    current_email_sender: str
    current_email_subject: str
    current_email_message_id: str
    classification: str
    total_emails: int
    email_index: int

**FINETUNING THE BASE MODEL**

In [ ]:
EXTRACTION_SYSTEM_PROMPT = "You are an expert at extracting tasks and deadlines. Respond with a JSON object containing 'task' and 'deadline' keys, or 'null'."

CLASSIFICATION_SYSTEM_PROMPT = "You are an email classification expert. Your job is to classify the email's intent. Choose ONLY one of the following categories: deadline, question, other."

def format_prompt_for_training(sample: dict) -> str:
    """
    A smart formatting function that uses the correct system prompt
    based on the type of response in the dataset.
    """
    response = sample['response']
    system_prompt = EXTRACTION_SYSTEM_PROMPT if response.startswith('{') else CLASSIFICATION_SYSTEM_PROMPT

    return f"""<|system|>
{system_prompt}<|end|>
<|user|>
{sample['text']}<|end|>
<|assistant|>
{response}<|end|>"""

def create_classification_prompt(email_content: str) -> str:
    """Creates the prompt for the classification tool, now using a consistent system message."""
    return f"""<|system|>
{CLASSIFICATION_SYSTEM_PROMPT}<|end|>
<|user|>
{email_content}<|end|>
<|assistant|>
"""

def create_extraction_prompt(email_content: str) -> str:
    """Creates the prompt for the extraction tool, now using a consistent system message."""
    return f"""<|system|>
{EXTRACTION_SYSTEM_PROMPT}<|end|>
<|user|>
{email_content}<|end|>
<|assistant|>
"""

In [33]:
def run_fine_tuning():
    """This function fine-tunes the model on the combined dataset."""
    print("🚀 Starting multi-task fine-tuning process...")
    MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
    DATASET_PATH = "dataset.jsonl"
    ADAPTERS_PATH = "./phi3-mini-deadline-extractor-adapters"

    if not os.path.exists(DATASET_PATH):
        raise FileNotFoundError(f"{DATASET_PATH} not found. Please upload your training data.")
    dataset = load_dataset("json", data_files=DATASET_PATH, split="train")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
    base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, quantization_config=quant_config, device_map="auto", trust_remote_code=True)
    base_model.config.use_cache = False

    peft_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules="all-linear")

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=6,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        logging_steps=5,
        learning_rate=2e-4,
        fp16=True,
        max_grad_norm=0.3,
        warmup_ratio=0.03,
        lr_scheduler_type="constant",
        report_to="none",
    )

    trainer = SFTTrainer(
        model=base_model,
        train_dataset=dataset,
        peft_config=peft_config,
        formatting_func=format_prompt_for_training,
        args=training_args,
    )

    print("⏳ Training model on both tasks...")
    trainer.train()
    print("✅ Fine-tuning complete.")
    trainer.model.save_pretrained(ADAPTERS_PATH)
    print(f"✅ Multi-task adapters saved to {ADAPTERS_PATH}")


run_fine_tuning()

🚀 Starting multi-task fine-tuning process...


Generating train split: 0 examples [00:00, ? examples/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Applying formatting function to train dataset:   0%|          | 0/38 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/38 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/38 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/38 [00:00<?, ? examples/s]

⏳ Training model on both tasks...


Step,Training Loss
5,1.710800
10,0.764700
15,0.459800
20,0.407100
25,0.321500
30,0.259500
35,0.173400
40,0.189200
45,0.106800
50,0.099900


✅ Fine-tuning complete.
✅ Multi-task adapters saved to ./phi3-mini-deadline-extractor-adapters


**AUTHENTICATION**

In [34]:
# --- GLOBAL VARIABLES ---
gmail_service = None
calendar_service = None
model = None
tokenizer = None

# Import Request for Google Auth
from google.auth.transport.requests import Request

def initialize_globals():
    """Authenticates and loads all necessary services and the model into global variables."""
    global gmail_service, calendar_service, model, tokenizer

    # --- Authenticate with Google ---
    print("🔑 Authenticating Google Services...")
    SCOPES = ['https://www.googleapis.com/auth/gmail.modify', 'https://www.googleapis.com/auth/calendar.events']
    TOKEN_FILE = 'token.json'
    CREDS_FILE = 'credentials.json'
    creds = None
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDS_FILE, SCOPES)
            creds = flow.run_console()
        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())
    gmail_service = build("gmail", "v1", credentials=creds)
    calendar_service = build("calendar", "v3", credentials=creds)
    print("✅ Google Services are ready.")

    # --- Load Fine-Tuned Model ---
    print("🧠 Loading fine-tuned model...")
    MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
    ADAPTERS_PATH = "./phi3-mini-deadline-extractor-adapters"
    if not os.path.exists(ADAPTERS_PATH):
        raise FileNotFoundError(f"Adapter path {ADAPTERS_PATH} not found. Run fine-tuning first.")

    quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
    base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, quantization_config=quant_config, device_map="auto", trust_remote_code=True)
    model = PeftModel.from_pretrained(base_model, ADAPTERS_PATH)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ Model is loaded and ready.")

# --- Run the initialization once at the start ---
initialize_globals()

🔑 Authenticating Google Services...
✅ Google Services are ready.
🧠 Loading fine-tuned model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model is loaded and ready.


***FUNCTIONS TO READ, EDIT , SEND MAILS and CREATE EVENTS IN GOOGLE CALENDAR***

In [35]:

def _create_and_send_reply(to: str, subject: str, in_reply_to: str, reply_body: str):
    """Creates an email message and sends it via the Gmail API."""
    print("   - Composing reply...")
    message = MIMEText(reply_body)
    message['to'] = to
    message['from'] = 'me'
    message['subject'] = f"Re: {subject}"
    message['In-Reply-To'] = in_reply_to
    message['References'] = in_reply_to
    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    body = {'raw': encoded_message}
    try:
        sent_message = gmail_service.users().messages().send(userId='me', body=body).execute()
        print(f"   - ✅ Success! Reply sent with ID: {sent_message['id']}")
    except HttpError as error:
        print(f"   - ❌ An API error occurred while sending the reply: {error}")

def draft_and_send_reply(state: AgentState) -> AgentState:
    """Tool 2: Drafts AND SENDS a reply for emails classified as questions."""
    print("🛠️ Using Tool: Draft and Send Reply")

    # Define the reply body
    reply_body = "Thank you for your question. I am an AI assistant and have forwarded your request to the appropriate team member. They will get back to you shortly."
    print(f"   - Sending Reply: '{reply_body}'")

    # ** FIX: Call the new helper to actually send the email **
    _create_and_send_reply(
        to=state["current_email_sender"],
        subject=state["current_email_subject"],
        in_reply_to=state["current_email_message_id"],
        reply_body=reply_body
    )

    _mark_email_as_read(state["current_email_id"])
    return {}


In [36]:
def _mark_as_processed_and_read(msg_id: str):
    """Applies the 'Processed' label and marks the email as read."""
    try:
        label_name = "Agent-Processed"
        # Ensure the label exists, create if not (optional, good for robustness)
        labels = gmail_service.users().labels().list(userId='me').execute()
        label_ids = [l['id'] for l in labels['labels'] if l['name'] == label_name]
        if not label_ids:
            label = gmail_service.users().labels().create(userId='me', body={'name': label_name}).execute()
            label_id = label['id']
        else:
            label_id = label_ids[0]

        gmail_service.users().messages().modify(
            userId='me',
            id=msg_id,
            body={'addLabelIds': [label_id], 'removeLabelIds': ['UNREAD']}
        ).execute()
        print(f"   - Marked email {msg_id} as processed and read.")
    except HttpError as e:
        print(f"   - ❌ ERROR: Failed to modify email {msg_id}: {e}")

In [37]:
def _label_as_processed(msg_id: str):
    """Applies the 'Processed' label but leaves the email unread."""
    try:
        label_name = "Agent-Processed"
        labels = gmail_service.users().labels().list(userId='me').execute()
        label_ids = [l['id'] for l in labels['labels'] if l['name'] == label_name]
        if not label_ids:
            label = gmail_service.users().labels().create(userId='me', body={'name': label_name}).execute()
            label_id = label['id']
        else:
            label_id = label_ids[0]

        gmail_service.users().messages().modify(
            userId='me',
            id=msg_id,
            body={'addLabelIds': [label_id]}
        ).execute()
        print(f"   - Marked email {msg_id} as processed (remains unread).")
    except HttpError as e:
        print(f"   - ❌ ERROR: Failed to apply label to {msg_id}: {e}")

In [38]:
def ignore_email(state: AgentState) -> AgentState:
    """
    NEW TOOL: Ignores the email by applying the processed label but leaving it unread.
    """
    print("   - Action: Ignoring Email")
    print(f"   - Email {state['current_email_id']} is not a deadline or question. Leaving unread.")
    _label_as_processed(state["current_email_id"]) # Apply label to prevent re-processing
    return {}

In [39]:

def get_email_details(msg_id: str) -> dict:
    """Fetches the content and key headers of a single email, returning them in a dictionary."""
    try:
        msg = gmail_service.users().messages().get(userId="me", id=msg_id, format='full').execute()
        headers = msg['payload'].get("headers", [])

        subject = next((h['value'] for h in headers if h['name'].lower() == 'subject'), "No Subject")
        sender_header = next((h['value'] for h in headers if h['name'].lower() == 'from'), "")
        message_id = next((h['value'] for h in headers if h['name'].lower() == 'message-id'), "")

        sender_email_match = re.search(r'<(.+?)>', sender_header)
        if sender_email_match:
            sender = sender_email_match.group(1)
        else:
            sender = sender_header # Fallback for simple email addresses

        body = ""
        if "parts" in msg['payload']:
            for part in msg['payload']['parts']:
                if part['mimeType'] == 'text/plain':
                    body_data = part['body'].get('data', '')
                    body = base64.urlsafe_b64decode(body_data.encode('ASCII')).decode('utf-8')
                    break
            if not body:
                 for part in msg['payload']['parts']:
                    if part['mimeType'] == 'text/html':
                        html_data = part['body'].get('data', '')
                        html_body = base64.urlsafe_b64decode(html_data.encode('ASCII')).decode('utf-8')
                        body = html2text.html2text(html_body)
                        break
        elif msg['payload'].get('body', {}).get('data'):
            body_data = msg['payload']['body']['data']
            body = base64.urlsafe_b64decode(body_data.encode('ASCII')).decode('utf-8')

        return {
            "body": f"Subject: {subject}\n\n{body.strip()}",
            "sender": sender,
            "subject": subject,
            "message_id": message_id
        }
    except HttpError as e:
        print(f"ERROR fetching email details: {e}")
        return {}

def _create_calendar_event(task: str, deadline_str: str) -> bool:
    """Creates a calendar event and returns True on success, False on failure."""
    try:
        deadline_dt = datetime.fromisoformat(deadline_str)

        event = {
            "summary": task,
            "start": {"dateTime": deadline_dt.isoformat(), "timeZone": "Asia/Kolkata"},
            "end": {"dateTime": (deadline_dt + timedelta(hours=1)).isoformat(), "timeZone": "Asia/Kolkata"},
        }
        created_event = calendar_service.events().insert(calendarId="primary", body=event).execute()

        if created_event and 'htmlLink' in created_event:
            print(f"   - ✅ Success! Calendar event created: {created_event.get('htmlLink')}")
            return True # Signal success
        else:
            print("   - ⚠️ API call succeeded but no event was created.")
            return False # Signal failure

    except ValueError:
        print(f"   - ❌ Validation Error: AI generated an invalid date format: '{deadline_str}'")
        return False # Signal failure
    except HttpError as error:
        print(f"   - ❌ An API error occurred: {error}")
        return False # Signal failure

def _mark_email_as_read(msg_id: str):
    """Marks an email as read with robust error handling."""
    try:
        response = gmail_service.users().messages().modify(userId='me', id=msg_id, body={'removeLabelIds': ['UNREAD']}).execute()
        if response and 'labelIds' in response and 'UNREAD' not in response['labelIds']:
             print(f"   - Marked email {msg_id} as read.")
    except HttpError as e:
        print(f"   - ❌ ERROR: Failed to mark email {msg_id} as read: {e}")

def fetch_unread_emails(state: AgentState) -> AgentState:
    """Fetches new emails and initializes the counters."""
    query = "is:unread -label:Agent-Processed category:primary"
    results = gmail_service.users().messages().list(userId="me", q=query).execute()
    messages = results.get("messages", [])

    # Initialize state for the run
    num_messages = len(messages)
    print(f"📬 Found {num_messages} new emails to process.")
    return {
        "unread_emails": messages,
        "total_emails": num_messages,
        "email_index": 0 # Start counter at 0
    }

def select_next_email(state: AgentState) -> AgentState:
    """Selects the next email and prints a clear header."""
    unread_emails = state.get("unread_emails", [])
    if not unread_emails:
        return {"current_email_id": None}

    # Increment counter and select next email
    email_index = state.get("email_index", 0) + 1
    total_emails = state.get("total_emails", 0)

    next_email = unread_emails.pop(0)
    email_id = next_email['id']
    details = get_email_details(email_id)

    print("\n" + "="*40)
    print(f"📧 PROCESSING EMAIL {email_index} of {total_emails}")
    print("="*40)
    print(f"   Subject: {details.get('subject')}")

    return {
        "unread_emails": unread_emails,
        "current_email_id": email_id,
        "current_email_content": details.get("body"),
        "current_email_sender": details.get("sender"),
        "current_email_subject": details.get("subject"),
        "current_email_message_id": details.get("message_id"),
        "email_index": email_index
    }

def classify_email(state: AgentState) -> AgentState:
    print("Thinking...")
    prompt = create_classification_prompt(state["current_email_content"])
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    prompt_len = inputs['input_ids'].shape[1]

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=10, eos_token_id=tokenizer.eos_token_id, use_cache=False)

    classification = tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True).strip().lower()
    # Clean up messy classification output
    if ":" in classification:
        classification = classification.split(":")[0]

    print(f"   - Classification: {classification}")
    return {"classification": classification}

def extract_and_create_event(state: AgentState) -> AgentState:
    """
    Tool 1: Extracts deadline, validates it, creates event, and ONLY marks read on success.
    This version includes robust error handling to prevent crashes.
    """
    print("🛠️ Using Tool: Create Calendar Event")
    prompt = create_extraction_prompt(state["current_email_content"])

    # --- 1. Generate text from the AI Model ---
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    prompt_len = inputs['input_ids'].shape[1]
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, eos_token_id=tokenizer.eos_token_id, use_cache=False)
    answer = tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True).strip()
    print(f"   - Model extracted: {answer}")

    # --- 2. Safely process the output ---
    success = False
    try:
        # ** FIX: The 'result' variable is created and used *inside* this safe block **
        result = json.loads(answer)
        if result and "task" in result and "deadline" in result:
            # The helper function now tells us if it succeeded
            success = _create_calendar_event(result["task"], result["deadline"])

    except (json.JSONDecodeError, TypeError):
        # This block runs if the AI's output is not a valid JSON, preventing the crash.
        print(f"   - ⚠️ Could not parse JSON from model output.")

    # --- 3. Only mark as read if the entire process was successful ---
    if success:
        _mark_as_processed_and_read(state["current_email_id"])
    else:
        print("   - ⚠️ Process failed. The original email will NOT be touched.")

    return {}

def draft_and_send_reply(state: AgentState) -> AgentState:
    """Tool 2: Drafts AND SENDS a reply for emails classified as questions."""
    print("🛠️ Using Tool: Draft and Send Reply")

    # 1. Create the new prompt to ask the AI for a reply
    prompt = create_reply_prompt(state["current_email_content"])
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    prompt_len = inputs['input_ids'].shape[1]

    # 2. Generate the reply using the AI model
    print("   - Thinking of a reply...")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=150, eos_token_id=tokenizer.eos_token_id, use_cache=False)

    # 3. Decode the generated text
    reply_body = tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True).strip()

    print(f"   - Sending Generated Reply: '{reply_body}'")

    # 4. Send the dynamically generated email
    _create_and_send_reply(
        to=state["current_email_sender"],
        subject=state["current_email_subject"],
        in_reply_to=state["current_email_message_id"],
        reply_body=reply_body
    )

    _mark_email_as_read(state["current_email_id"])
    return {}

def archive_email(state: AgentState) -> AgentState:
    print(" Using Tool: Archive Email")
    print(f"   - Archiving email {state['current_email_id']}.")
    _mark_as_processed_and_read(state["current_email_id"])
    return {}


In [40]:
def create_reply_prompt(email_content: str) -> str:
    """Creates a prompt for the LLM to draft a helpful reply."""
    return f"""<|system|>
You are a helpful AI assistant. Read the user's email and provide a concise and helpful response to their question.<|end|>
<|user|>
{email_content}<|end|>
<|assistant|>
"""

In [41]:

def _create_draft(to: str, subject: str, in_reply_to: str, reply_body: str):
    """Creates a draft reply in Gmail for human review."""
    print("   - Composing draft...")
    message = MIMEText(reply_body)
    message['to'] = to
    message['from'] = 'me'
    message['subject'] = f"Re: {subject}"
    message['In-Reply-To'] = in_reply_to
    message['References'] = in_reply_to

    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    body = {'message': {'raw': encoded_message}}

    try:
        draft = gmail_service.users().drafts().create(userId='me', body=body).execute()
        print(f"   - ✅ Success! Draft created with ID: {draft['id']}. Please review it in Gmail.")
    except HttpError as error:
        print(f"   - ❌ An API error occurred while creating the draft: {error}")

In [42]:
def draft_reply_for_approval(state: AgentState) -> AgentState:
    """Tool 2: Drafts a reply for human review and approval."""
    print("🛠️ Using Tool: Draft Reply for Approval")

    # 1. Create the prompt to ask the AI for a reply
    prompt = create_reply_prompt(state["current_email_content"])
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    prompt_len = inputs['input_ids'].shape[1]

    # 2. Generate the reply using the AI model
    print("   - Thinking of a reply...")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=150, eos_token_id=tokenizer.eos_token_id, use_cache=False)

    # 3. Decode the generated text
    reply_body = tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True).strip()

    print(f"   - Generated Draft: '{reply_body}'")

    # 4. Create the draft in Gmail instead of sending
    _create_draft(
        to=state["current_email_sender"],
        subject=state["current_email_subject"],
        in_reply_to=state["current_email_message_id"],
        reply_body=reply_body
    )
    # ** FIX: Apply the 'Processed' label so we don't draft a reply again **
    _label_as_processed(state["current_email_id"])
    return {}

**THE AGENT**

In [43]:

workflow = StateGraph(AgentState)

workflow.add_node("fetch_emails", fetch_unread_emails)
workflow.add_node("select_next_email", select_next_email)
workflow.add_node("classify_email", classify_email)
workflow.add_node("create_event_tool", extract_and_create_event)
workflow.add_node("draft_reply_tool", draft_reply_for_approval)
workflow.add_node("ignore_tool", ignore_email)


# This function is the "brain" of our agent, deciding what to do next.
def router(state: AgentState):
    print("🧠 Agent is making a decision...")
    classification = state.get("classification", "").lower()
    if "deadline" in classification:
        return "create_event_tool"
    elif "question" in classification:
        return "draft_reply_tool"
    else:
        return "ignore_tool"

# 4. Define the edges and control flow
workflow.set_entry_point("fetch_emails")
workflow.add_edge("fetch_emails", "select_next_email")

# This conditional edge determines if the loop should continue or end
def should_continue(state: AgentState):
    if state.get("current_email_id") is None:
        return "end"
    else:
        return "classify_email"

workflow.add_conditional_edges(
    "select_next_email",
    should_continue,
    {
        "end": END,
        "classify_email": "classify_email"
    }
)

# This is the main decision-making edge, powered by our router
workflow.add_conditional_edges(
    "classify_email",
    router,
    {
        "create_event_tool": "create_event_tool",
        "draft_reply_tool": "draft_reply_tool",
        "ignore_tool": "ignore_tool",
    }
)

# After a tool is used, the agent loops back to process the next email
workflow.add_edge("create_event_tool", "select_next_email")
workflow.add_edge("draft_reply_tool", "select_next_email")
workflow.add_edge("ignore_tool", "select_next_email")

# 5. Compile and run the agent
app = workflow.compile()
print("🚀 Agent compiled. Starting main loop...")

while True:
    try:
        print("\n\n" + "="*60)
        print(f"🚀 STARTING AGENT RUN at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*60)

        app.invoke({})

        print("\n" + "="*60)
        print("✅ Run Complete. Agent is now sleeping.")
        print("="*60)
        time.sleep(30)

    except KeyboardInterrupt:
        print("\n🚫 Agent run manually stopped. Exiting.")
        break
    except Exception as e:
        print(f"💥 A critical error occurred: {e}")
        time.sleep(300)

🚀 Agent compiled. Starting main loop...


🚀 STARTING AGENT RUN at 2025-09-17 11:38:53
📬 Found 2 new emails to process.

📧 PROCESSING EMAIL 1 of 2
   Subject: 
Thinking...
   - Classification: question
🧠 Agent is making a decision...
🛠️ Using Tool: Draft Reply for Approval
   - Thinking of a reply...
   - Generated Draft: 'I'm sorry to hear that you're having trouble with the assignment. Here are a few resources that might help:

1. Check the course syllabus for any additional resources or office hours.
2. Reach out to your instructor or TA for guidance.
3. Look for online forums or discussion boards related to the course.
4. Consider hiring a tutor or joining a study group.

Good luck, and let me know if there's anything else I can do for you!'
   - Composing draft...
   - ✅ Success! Draft created with ID: r-4645681052269242232. Please review it in Gmail.
   - Marked email 199576f88f7d913f as processed (remains unread).

📧 PROCESSING EMAIL 2 of 2
   Subject: next meeting
Thinking...
  

In [50]:
!pip install -q scikit-learn sentence-transformers

import json
import os
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import f1_score

def run_evaluation():
    """
    Runs an advanced evaluation, calculating F1 Score and Task Success Rate.
    """
    print("🚀 Starting Advanced Evaluation...")

    # --- 1. Load Models and Test Data ---
    print("   - Loading similarity model...")
    try:
        similarity_model = SentenceTransformer('all-MiniLM-L6-v2')
    except Exception as e:
        print(f"   - ❌ Could not load similarity model. Please check your connection. Error: {e}")
        return
    print("   - Model loaded.")

    test_file = "test_emails.jsonl"
    if not os.path.exists(test_file):
        print(f"❌ Evaluation file not found: {test_file}")
        return

    test_data = []
    with open(test_file, 'r') as f:
        for line in f:
            test_data.append(json.loads(line))

    # --- 2. Initialize Metrics and Storage for Scores ---
    successful_tasks = 0
    ground_truth_classifications = []
    predicted_classifications = []

    deadline_emails_total = 0
    deadline_extraction_success = 0
    deadline_values_correct = 0

    question_emails_total = 0
    similarity_scores = []

    # --- 3. Loop Through All Test Emails ---
    for item in tqdm(test_data, desc="Evaluating Emails"):
        email_text = item['text']
        ground_truth = item['ground_truth']
        task_is_successful = False

        # --- Test Classification ---
        prompt = create_classification_prompt(email_text)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        prompt_len = inputs['input_ids'].shape[1]
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=10, eos_token_id=tokenizer.eos_token_id, use_cache=False)

        prediction = tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True).strip().lower()

        # Clean the prediction for accurate scoring
        if "deadline" in prediction: predicted_class = "deadline"
        elif "question" in prediction: predicted_class = "question"
        else: predicted_class = "other"

        ground_truth_classifications.append(ground_truth['classification'])
        predicted_classifications.append(predicted_class)
        is_classification_correct = (predicted_class == ground_truth['classification'])

        # --- Test Tools and Determine End-to-End Task Success ---
        if ground_truth['classification'] == 'deadline':
            deadline_emails_total += 1
            prompt = create_extraction_prompt(email_text)
            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
            prompt_len = inputs['input_ids'].shape[1]
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=100, eos_token_id=tokenizer.eos_token_id, use_cache=False)
            extracted_text = tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True).strip()

            try:
                extracted_json = json.loads(extracted_text)
                if isinstance(extracted_json, dict):
                    deadline_extraction_success += 1
                    if (extracted_json.get('task') == ground_truth.get('task') and extracted_json.get('deadline') == ground_truth.get('deadline')):
                        deadline_values_correct += 1
                        if is_classification_correct:
                            task_is_successful = True
            except (json.JSONDecodeError, TypeError):
                pass

        elif ground_truth['classification'] == 'question':
            question_emails_total += 1
            reply_prompt = create_reply_prompt(email_text)
            inputs = tokenizer(reply_prompt, return_tensors="pt").to(model.device)
            prompt_len = inputs['input_ids'].shape[1]
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=150, eos_token_id=tokenizer.eos_token_id, use_cache=False)
            agent_reply = tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True).strip()
            golden_reply = ground_truth.get("golden_reply")
            if agent_reply and golden_reply:
                embedding1 = similarity_model.encode(agent_reply, convert_to_tensor=True)
                embedding2 = similarity_model.encode(golden_reply, convert_to_tensor=True)
                cosine_score = util.cos_sim(embedding1, embedding2).item()
                similarity_scores.append(cosine_score)
                if is_classification_correct and cosine_score > 0.75: # Success threshold
                    task_is_successful = True

        elif ground_truth['classification'] == 'other':
            if is_classification_correct:
                task_is_successful = True

        if task_is_successful:
            successful_tasks += 1

    # --- 4. Calculate and Print Final Results ---
    print("\n" + "="*50)
    print("📊 ADVANCED EVALUATION RESULTS")
    print("="*50)

    # F1 Score for Classification
    f1 = f1_score(ground_truth_classifications, predicted_classifications, average="weighted")
    print(f"\nRouting Performance (Classification):")
    print(f"  - F1 Score (Weighted): {f1:.2f}")
    print("     (A score combining precision and recall. 1.0 is perfect.)")

    # Task Success Rate (End-to-End)
    task_success_rate = (successful_tasks / len(test_data)) * 100
    print(f"\nAgent End-to-End Performance:")
    print(f"  - Task Success Rate: {task_success_rate:.1f}% ({successful_tasks} of {len(test_data)} tasks completed perfectly)")
    print("     (Measures if the agent did the right thing from start to finish.)")

    # Semantic Similarity for Replies
    if question_emails_total > 0:
        avg_similarity = (sum(similarity_scores) / len(similarity_scores)) * 100 if similarity_scores else 0
        print(f"\nReply Generation Quality (for 'question' emails):")
        print(f"  - Average Semantic Similarity: {avg_similarity:.2f}%")
        print("     (Measures how close in meaning replies are to the ideal answers.)")

    print("\n" + "="*50)

# --- Run the evaluation ---
# Ensure your model, tokenizer, and prompt functions are defined before running this.
run_evaluation()

🚀 Starting Advanced Evaluation...
   - Loading similarity model...
   - Model loaded.


Evaluating Emails: 100%|██████████| 15/15 [03:23<00:00, 13.57s/it]


📊 ADVANCED EVALUATION RESULTS

Routing Performance (Classification):
  - F1 Score (Weighted): 0.68
     (A score combining precision and recall. 1.0 is perfect.)

Agent End-to-End Performance:
  - Task Success Rate: 46.7% (7 of 15 tasks completed perfectly)
     (Measures if the agent did the right thing from start to finish.)

Reply Generation Quality (for 'question' emails):
  - Average Semantic Similarity: 63.64%
     (Measures how close in meaning replies are to the ideal answers.)

